# plot_vorticity.ipynb

Author: Robert M. Frost

University of Oklahoma

Created: 08 February 2024

Purpose: Read in all cases to plot vorticity

In [1]:
import sys
sys.path.append("/home/rfrost/LES-utils/")

import seaborn
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import MultipleLocator
from LESutils import load_full
import seaborn
import cmocean
from matplotlib.gridspec import GridSpec
from matplotlib.colors import ListedColormap

### Settings

In [2]:
# directory to save figures
figdir = "/home/rfrost/manuscript_plots/"
# simulation to analyze
sims = ["full_step_6", "full_step_9", "full_step_12", "full_step_15"]

# z/zi to plot
height = 0.25

# start and end timesteps
t0 = 576000
t1 = 1152000
# start and end in hours
t0hr = t0 * 0.05 / 3600
t1hr = t1 * 0.05 / 3600
# spacing of output
dt = 1000

# lists to store stats, roll factor, and length scales
s_all, v_all = [], []
# loop over sims
for sim in sims:
    dnc = f"/home/rfrost/simulations/nc/{sim}/"
    # volumetric stats
    s = xr.open_dataset(f"{dnc}{t0}_{t1}_stats.nc")
    # convert time to hours
    s["time"] = s.time / 3600 + t0hr
    s_all.append(s)

    # vorticity
    v = xr.open_dataset(f"{dnc}{t0}_{t1}_vorticity.nc")
    # convert time to hours
    v["time"] = s.time / 3600 + t0hr
    v_all.append(s)

FileNotFoundError: [Errno 2] No such file or directory: b'/home/rfrost/simulations/nc/full_step_6/576000_1152000_vorticity.nc'

In [ ]:
# plotting setup

# Set the font weight for plot titles
plt.rcParams['axes.titleweight'] = 'bold'

# Set the font weight for x-axis and y-axis labels
plt.rcParams['axes.labelweight'] = 'normal'

plt.rcParams['text.latex.preamble'] = r'\usepackage{bm}'

# Other plotting setup
rc('font', family='sans-serif')
rc('font', weight='normal', size=20)
rc('figure', facecolor='white')

# lists/cmaps for titles and stuff
ug_list = ["6", "9", "12", "15"]
time_colors = seaborn.color_palette("cubehelix", 7)
ug_colors = seaborn.color_palette("flare", 4)
linestyle = ["--","solid","solid","-.","-."]

In [ ]:
# find average T_L at hr 9.8333
T_L = (843.69 + 861.95 + 841.57 + 844.23) / 4

tb = -2
tc = 4.5
td = 11
te = 17.5
tf = 24

tlmax = max((s_all[0].time-10)*3600/T_L)
tlmin = min((s_all[0].time-10)*3600/T_L)

# normalized times to be plot
jttl = [-2, 4.5, 11, 17.5, 24]
jtall = [abs(((s_all[0].time.values-10)*3600)/T_L - jttl[jt]).argmin() for jt in range(len(jttl))]

## Plotting

### End member cross sections

In [ ]:
fig = plt.figure(figsize=(17,13))

gs1 = GridSpec(2,3, top=0.95, bottom=0.275)
ax1 = fig.add_subplot(gs1[0,0])
ax2 = fig.add_subplot(gs1[1,0])
ax3 = fig.add_subplot(gs1[0,1])
ax4 = fig.add_subplot(gs1[1,1])
ax5 = fig.add_subplot(gs1[0,2])
ax6 = fig.add_subplot(gs1[1,2])
gs2 = GridSpec(1,1, top=0.20, bottom=0.175, left=0.125, right=0.625)
ax7 = fig.add_subplot(gs2[:])
gs3 = GridSpec(1,1, top=0.20, bottom=0.175, left=0.68, right=0.9)
ax8 = fig.add_subplot(gs3[:])

x, y = s_all[0].x/1000, s_all[0].y/1000

clevs = [-4.  , -3.75, -3.5 , -3.25, -3.  , -2.75, -2.5 , -2.25, -2.  ,
       -1.75, -1.5 , -1.25, -1.  , -0.75, -0.5 , -0.25  ,  0.25,
        0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,  2.25,  2.5 ,
        2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.]
clevw = np.linspace(-2,2,51)
cmapw = cmocean.cm.curl
cmap = mpl.cm.seismic

cf = ax1.contourf(x, y, s_all[0].zeta[359,:,:,0].rolling(x=3,y=3).mean().T * 10 ** 2, clevs, cmap=cmap, extend="both")
ax1.text(-3.5, 6, "$U_g = 6$ m s$^{-1}$", rotation=90, va="center", ha="center")
ax1.set_title("$\zeta_{z = 6.25 \> m}$")

ax2.contourf(x, y, s_all[3].zeta[132,:,:,0].rolling(x=3,y=3).mean().T * 10 ** 2, clevs, cmap=cmap, extend="both")
ax2.text(-3.5, 6, "$U_g = 15$ m s$^{-1}$", rotation=90, va="center", ha="center")

ax3.contourf(x, y, s_all[0].zeta[359,:,:,8].rolling(x=3,y=3).mean().T * 10 ** 2, clevs, cmap=cmap, extend="both")
ax3.set_title("$\zeta_{z = 107 \> m}$")
ax4.contourf(x, y, s_all[3].zeta[132,:,:,8].rolling(x=3,y=3).mean().T * 10 ** 2, clevs, cmap=cmap, extend="both")

cfw = ax5.contourf(x, y, s_all[0].w[359,:,:,16].T / s_all[0].wstar[359], clevw, cmap=cmapw, extend="both")
ax5.set_title("$w_{z = 208 \> m}$")
ax6.contourf(x, y, s_all[3].w[132,:,:,16].T / s_all[3].wstar[132], clevw, cmap=cmapw, extend="both")

cbar = plt.colorbar(cf, ax7, orientation="horizontal")
cbar.set_label("$\zeta$ [$10^{-2}$ s$^{-1}$]")
cbar.set_ticks(np.arange(-4, 4.1, 1))

cbar = plt.colorbar(cfw, ax8, orientation="horizontal")
cbar.set_label("$w / w_*$")
cbar.set_ticks(np.arange(-2,2.1,1))

axes = [ax1, ax2, ax3, ax4, ax5, ax6]
for ax in axes:
    ax.xaxis.set_major_locator(MultipleLocator(2))
    ax.yaxis.set_major_locator(MultipleLocator(2))
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.yaxis.set_minor_locator(MultipleLocator(1))
    # ax.set_xlim(4,8)
    # ax.set_ylim(4,8)

ax1.set_ylabel("$y$ [km]")
ax2.set_ylabel("$y$ [km]")
ax2.set_xlabel("$x$ [km]")
ax4.set_xlabel("$x$ [km]")
ax6.set_xlabel("$x$ [km]")